# optionm'

In [ ]:
open rust
open rust_operators

In [ ]:
//// test

open testing

## optionm'

### default_value

In [ ]:
inl default_value d =
    optionm.defaultWith d

In [ ]:
//// test

None
|> default_value 3i32
|> _assert_eq 3i32

{ name = __assert_eq; expected = 3 }


### (/??)

In [ ]:
inl (/??) a b =
    a |> default_value b

In [ ]:
//// test

None /?? 3i32
|> _assert_eq 3i32

{ name = __assert_eq; expected = 3 }


### default_with

In [ ]:
inl default_with fn = function
    | Some x => x
    | None => fn ()

In [ ]:
//// test

None
|> default_with fun () => 3i32
|> _assert_eq 3i32

{ name = __assert_eq; expected = 3 }


### choose

In [ ]:
inl choose fn a b =
    match a, b with
    | Some x, Some y => fn x y |> Some
    | _ => None

In [ ]:
//// test

(Some 2i32, Some 3)
||> choose (+)
|> _assert_eq (Some 5)

{ name = __assert_eq; expected = US0_0 5 }


### iter

In [ ]:
inl iter fn = function
    | Some x => fn x
    | None => ()

In [ ]:
//// test

inl n = mut 1i32
inl fn =
    fun n' =>
        n <- *n + n'
Some 1i32 |> iter fn
None |> iter fn
*n
|> _assert_eq 2i32

{ name = __assert_eq; expected = 2 }


### flatten

In [ ]:
inl flatten x =
    match x with
    | Some (Some x) => Some x
    | _ => None

## fsharp

### option'

In [ ]:
inl option'_types () : () =
    backend_switch {
        Gleam = fun () => global "import gleam/option"
    }
nominal option'_gleam t =
    `(
        option'_types ()
        $'' : $'option.Option(`t)'
    )
type option'_switch t =
    {
        Gleam : option'_gleam t
        Fsharp : $"`t option"
        Python : t
    }
nominal option' t = $'backend_switch `(option'_switch t)'

### none'

In [ ]:
inl none' forall t. () : option' t =
    backend_switch {
        Gleam = fun () => $'option.None' : option' t
        Fsharp = fun () => $'None' : option' t
        Python = fun () => $'None' : option' t
    }

### some'

In [ ]:
inl some' forall t. (x : t) : option' t =
    backend_switch {
        Gleam = fun () => $'!x |> option.Some' : option' t
        Fsharp = fun () => $'Some !x ' : option' t
        Python = fun () => $'!x \# some\' ' : option' t
    }

### default_value'

In [ ]:
inl default_value' forall t. (value : t) (x : option' t) : t =
    backend_switch {
        Gleam = fun () => $'!x |> option.unwrap(!value)' : t
        Fsharp = fun () => $'!x |> Option.defaultValue !value ' : t
        Python = fun () => $'!x or !value ' : t
    }

### get'

In [ ]:
inl get' forall t. (x : option' t) : t =
    backend_switch {
        Gleam = fun () =>
            option'_types ()
            $'!x |> option.unwrap (Nil)' : t
        Fsharp = fun () => $'!x |> Option.get' : t
        Python = fun () => $'!x ' : t
    }

### box

In [ ]:
inl box forall t. (x : option t) : option' t =
    // x
    // |> optionm.map some'
    // |> default_with none'
    match x with
    | Some x => some' x
    | None => none' ()

### map

In [ ]:
inl map forall t u. (fn : t -> u) (x : option' t) : option' u =
    backend_switch {
        Gleam = fun () =>
            option'_types ()
            // inl fn = join fn
            $'!x |> option.map(!fn)' : option' u
        Fsharp = fun () =>
            inl fn = join fn
            $'!x |> Option.map !fn ' : option' u
        Python = fun () =>
            if x =. none' ()
            then none' ()
            else fn $'!x ' |> fun x => $'!x ' : option' u
    }

### map''

In [ ]:
inl map'' forall t u. (fn : t -> u) (x : option' t) : option' u =
    x |> map fn

### unbox

In [ ]:
inl unbox forall t. (x : option' t) : option t =
    x |> map'' Some |> default_value' None
    // inl some x : option t = Some x
    // inl some = join some
    // inl none : option t = None
    // $'!x |> Option.map !some |> Option.defaultValue !none '

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

inl x = Some 3i32
inl y : option i32 = None
inl x' = x |> box |> unbox
inl y' = y |> box |> map id |> unbox
(x', y') |> _assert_eq' (x, y)

.py output (Python):
{ name = __assert_eq'; expected = US0_0(v0=3), US0_1() }

.rs output:
{ name = __assert_eq'; expected = US0_0(3), US0_1 }

.ts output:
{ name = __assert_eq'; expected = US0_0 3, US0_1 }

.py output:
{ name = __assert_eq'; expected = US0_0 3, US0_1 }

.gleam output (Gleam):
{ name = __assert_eq'; expected = Us0i0(3), Us0i1 }



.fsx output:
{ name = __assert_eq'; expected = US0_0 3, US0_1 }


### of_obj

In [ ]:
inl of_obj forall t. (x : t) : option' t =
    backend_switch {
        Gleam = fun () =>
            option'_types ()
            $'!x |> option.Some' : option' t
        Fsharp = fun () =>
            $'let mutable _!x = None'
            $'\#if \!FABLE_COMPILER && \!WASM && \!CONTRACT'
            ((x |> $'Option.ofObj') : option' t) |> emit_unit
            $'\#else'
            $'Some !x '
            $'\#endif'
            $'|> fun x -> _!x <- Some x'
            $'match _!x with Some x -> x | None -> failwith "optionm\'.of_obj / _!x=None"' : option' t
        Python = fun () =>
            $'!x ' : option' t
    }

In [ ]:
//// test
////! gleam // Expected type: #(String) Found type: #(Nil)
///! fsharp
///! cuda
////! rust // attempted to zero-initialize type `alloc::sync::Arc<dyn core::any::Any>`, which is invalid
///! typescript
///! python

null ()
|> of_obj
|> unbox
|> _assert_eq (None : option string)

.py output (Python):
{ name = __assert_eq; expected = US0_1() }

.ts output:
{ name = __assert_eq; expected = US0_1 }

.py output:
{ name = __assert_eq; expected = US0_1 }



.fsx output:
{ name = __assert_eq; expected = US0_1 }


In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

""
|> of_obj
|> unbox
|> _assert_eq' (Some "")

.py output (Python):
{ name = __assert_eq'; expected = US0_0(v0='') }

.rs output:
{ name = __assert_eq'; expected = US0_0("") }

.ts output:
{ name = __assert_eq'; expected = US0_0  }

.py output:
{ name = __assert_eq'; expected = US0_0 "" }

.gleam output (Gleam):
{ name = __assert_eq'; expected = Us0i0("") }



.fsx output:
{ name = __assert_eq'; expected = US0_0 "" }


### flatten'

In [ ]:
inl flatten' x =
    x
    |> unbox
    |> optionm.map unbox
    |> flatten

## rust

### try'

In [ ]:
inl try' forall t. (x : option' t) : t =
    !\\(x, $'"$0?"')

### map'

In [ ]:
inl map' forall t u. (fn : t -> u) (x : option' t) : option' u =
    (!\\(x, $'"true; let _optionm_map_ = $0.map(|x| { //"') : bool) |> ignore
    inl result = fn !\($'"x"')
    (!\\(result, $'"true; $0 })"') : bool) |> ignore
    !\($'"_optionm_map_"')

### unwrap

In [ ]:
inl unwrap forall t. (x : option' t) : t =
    !\\(x, $'"$0.unwrap()"')

### take

In [ ]:
inl take forall t. (x : option' t) : option' t =
    (!\\(x, $'"true; let mut !x = !x"') : bool) |> ignore
    !\\(x, $'"Option::take(&mut $0)"')

### take_ref

In [ ]:
inl take_ref forall t. (x : rust.ref (option' t)) : option' t =
    (!\\(x, $'"true; let mut !x = !x"') : bool) |> ignore
    !\\(x, $'"Option::take(&mut $0)"')

### take_ref_mut

In [ ]:
inl take_ref_mut forall t. (x : rust.ref (rust.mut' (option' t))) : option' t =
    !\\(x, $'"Option::take($0)"')

### cloned

In [ ]:
inl cloned forall t. (x : option' (rust.ref t)) : option' t =
    !\\(x, $'"$0.cloned()"')

### as_ref

In [ ]:
inl as_ref forall t. (x : rust.ref (option' t)) : option' (rust.ref t) =
    !\\(x, $'"$0.as_ref()"')

### as_mut

In [ ]:
inl as_mut forall t. (x : rust.ref (rust.mut' (option' t))) : option' (rust.ref (rust.mut' t)) =
    !\\(x, $'"$0.as_mut()"')

### unwrap_or

In [ ]:
inl unwrap_or forall t. (def : t) (x : option' t) : t =
    !\($'"!x.unwrap_or(!def)"')

### and_then

In [ ]:
inl and_then forall t u. (fn : t -> option' u) (x : option' t) : option' u =
    !\\((x, fn), $'"$0.and_then(|x| $1(x))"')

### rc_upgrade

In [ ]:
inl rc_upgrade forall t. (x : rust.weak_rc t) : option' (rust.rc t) =
    !\\(x, $'"std::rc::Weak::upgrade(&$0)"')

### rc_into_inner

In [ ]:
inl rc_into_inner forall t. (x : rust.rc t) : option' t =
    !\\(x, $'"std::rc::Rc::into_inner($0)"')

In [ ]:
//// test
///! rust

rust.new_rc 0i32
|> rc_into_inner
|> unbox
|> _assert_eq' (Some 0i32)

{ name = __assert_eq'; expected = US0_0(0) }
